In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.distributions import Normal as norm
from termcolor import colored
from sklearn.decomposition import PCA
from scipy import linalg as LA
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import multinomial
from numpy import genfromtxt
from Main_functions import *
import random

### Setting the data distribuation 

Pairs $(x_i,y_i)\in \mathbb{R}^d\times \mathbb{R}^d$ for $i=1, \ldots, N$ are generated i.i.d. from density distribuation $f(\cdot)\times f(\cdot)$.  

Parameters:

$d:$ dimention of data points

$N:$ number of pairs

$f(\cdot):$ density distribuation


### Given the eigenvalue of ground truth $M_t$, we randomly generate it.

In [ ]:
csv_path = 'CSVs/gaussian_laplace_'
image_path = 'Images/gaussian_laplace_'
title_suffix = '\n (Gaussian Noise Laplace model)'
noise_type = 'Gaussian'
model_type = 'Laplace'

In [ ]:
d = 10
rank = 5
seed = 2023

In [ ]:
np.random.seed(seed)
non_zero_eigens_of_M_star = np.round(np.random.uniform(low=0.0, high=1.0, size= rank), 2)
non_zero_eigens_of_M_star

In [ ]:
diag = np.zeros(d)
diag[:rank] = non_zero_eigens_of_M_star
diag

In [ ]:
data_model = DG(diag, seed = 22)

In [ ]:
data_model.M_diag

In [ ]:
data_model.M_t.shape, data_model.B_t.shape

### Given the eigenvalue of covariance matrix Cov, we randomly generate it.

In [ ]:
np.random.seed(2024)
diag_cov = np.round(np.random.uniform(low=0.0, high=1.0, size= d), 2)
diag_cov

In [ ]:
diag_cov.sum()

In [ ]:
Sigma, U_sigma = random_covariance(diag_cov, seed = 21)

In [ ]:
f = multivariate_normal(mean = np.zeros(d), cov = Sigma)

### What is the average of squared norm of points generating where $x\sim f(x)$ with $\mathbb{E}(x) = \mu$ and ${\rm Cov}(x) = \Sigma$

If $x\sim f(x)$ where $\mathbb{E}(x) = \mu$ and ${\rm Cov}(x) = \Sigma$, then 
\begin{align}
\mathbb{E}\left(x^\top Mx\right) & = \mathbb{E}\left({\rm tr}(x^\top Mx)\right)\\
 & = \mathbb{E}\left({\rm tr}(xx^\top M)\right)\\
 & = {\rm tr}\left(\mathbb{E}\left(xx^\top\right)M\right)\\
 & = {\rm tr}\left(\left[{\rm Cov}(x) - \mathbb{E}(x)\mathbb{E}(x)^\top\right]M\right)
\end{align}

If we set  $\mu = \mathbf{0}$, then 
$$\mathbb{E}\left(x^\top Mx\right) = {\rm tr}\left(\Sigma M\right).$$
Therefore, since $x$ and $y$ are independent, 
$$\mathbb{E}\left(\|x-y\|_M^2\right) = 2{\rm tr}\left(\Sigma M\right).$$

In [ ]:
snm = 2 * np.trace(data_model.M_t @ Sigma)
print("squared norm mean = {}".format(snm))

### Check the average empirically

In [ ]:
N = 10000
X_1, Y_1 = sample(f, N)
sn = (((X_1 - Y_1) @ data_model.B_t)**2).sum()/N
print("emperical squared norm mean = {}".format(sn))

### Set $\tau$ to generate data points (binary case!)

In [ ]:
s = 0.2

In [ ]:
std = s* np.pi/np.sqrt(3)
std

In [ ]:
tau = [1.3]
N = 20000

In [ ]:
X, Y, D_noisy, D_no_noisy = data_model.generate(f, N, tau = np.array(tau), 
                                          noise_type = noise_type, noise_par = std)

In [ ]:
noise_amount = (D_noisy != D_no_noisy).mean()
print('the amount of noise: {}'.format(noise_amount))

In [ ]:
for i in range(len(tau)+1):
    print('number of pairs with {} as their labels is {}'.format(i,(D_no_noisy==i).sum()))

In [ ]:
(D_no_noisy== 0).sum()/N

### Change to PyTorch tensor!

In [ ]:
X = torch.tensor(X, dtype = torch.float64)
Y = torch.tensor(Y, dtype = torch.float64)
D_noisy = torch.tensor(D_noisy, dtype = torch.torch.int64)
D_no_noisy = torch.tensor(D_no_noisy, dtype = torch.torch.int64)

In [ ]:
X, Y, D, X_T, Y_T, D_T, D_no_noise_T, X_test, Y_test, D_test, D_no_noise_test = train_test_split(X, 
                                                                                                 Y, 
                                                                                                 D_noisy, 
                                                                                                 D_no_noisy, 
                                                                                                 n_train = 15000)

In [ ]:
X.dtype, D.dtype

## Set the model and train it

In [ ]:
k = 10
n_labels = len(tau) + 1

In [ ]:
model = ML(d, k, n_labels, 
   X_T, Y_T, D_T, D_no_noise_T, 
   X_test, Y_test, D_test, D_no_noise_test, Type = model_type)

In [ ]:
model.train(learning_rate = 5e-1, 
            n_iters = 30001, 
            decay = .95,
            f = f, 
            B_star = data_model.B_t, 
            tau_star = data_model.tau_t, 
            N = 1000)
# N here is used to generate N samples to estimate L_1_f_norm

In [ ]:
X_new, Y_new, _, D_new = data_model.generate(f, 
                                             N = 10000, 
                                             tau = np.array(tau), 
                                             noise_type = None, 
                                             noise_par = None)

In [ ]:
X_new = torch.tensor(X_new, dtype = torch.float64)
Y_new = torch.tensor(Y_new, dtype = torch.float64)
D_new = torch.tensor(D_new, dtype = torch.torch.int64)

In [ ]:
ac_unseen = model.accuracy(X_new, Y_new, D_new)
print('accuracy on unseen data {}'.format(ac_unseen))

###  The variance of logistic distribution with scale $s$ is $\sigma^2 = \frac{s^2\pi^2}{3}$.

In [ ]:
# s_prime = np.array(tau)[0]/model.Tau.detach().numpy()[0]
# print(s_prime, s_star)

In [ ]:
# s = std*np.sqrt(3)/np.pi
# s

In [ ]:
M_hat = (model.B @ model.B.T).detach().numpy()*s
M_star = data_model.M_t
tau_hat = model.Tau.detach().numpy()[0]*s
tau_star = data_model.tau_t[0]

print((np.abs(M_hat - M_star)).max())

### Relative errors

$$\frac{\left\|\frac{\hat{M}}{\hat{\tau}} - \frac{M^*}{\tau^*}\right\|_2}{\left\|\frac{M^*}{\tau^*}\right\|_2}$$

In [ ]:
M_hat_normal = model.B @ model.B.T / model.Tau[0]
M_t_normal = data_model.M_t/tau[0]

In [ ]:
np.sqrt(np.sum((M_t_normal - M_hat_normal.detach().numpy())**2))/np.sqrt((M_t_normal**2).sum())

$$\left\|\frac{\hat{M}}{\hat{\tau}} - \frac{M^*}{\tau^*}\right\|_\infty$$

In [ ]:
(np.abs(M_t_normal - M_hat_normal.detach().numpy())).max()

$$\left|\frac{\hat{M}}{\hat{\tau}} - \frac{M^*}{\tau^*}\right|_{L_1(f)}$$

In [ ]:
L_1_f_norm(f, model.B.detach().numpy(), model.Tau.detach().numpy(), 
           data_model.B_t, data_model.tau_t, N = 10000)

$$\left|\hat{\tau} - \frac{\tau^*}{s}\right|$$

In [ ]:
np.abs(model.Tau.detach().numpy()[0] - data_model.tau_t[0]/s)

In [ ]:
(np.abs(M_t_normal - M_hat_normal.detach().numpy())).mean()

In [ ]:
(np.abs(M_t_normal - M_hat_normal.detach().numpy())).max()

In [ ]:
np.sqrt(np.sum((M_t_normal - M_hat_normal.detach().numpy())**2))/np.sqrt((M_t_normal**2).sum())

In [ ]:
np.sqrt((M_t_normal**2).sum())

In [ ]:
print(model.Tau.detach().numpy())
print(np.array(tau))
print(np.array(tau)[0]/model.Tau.detach().numpy()[0], s, tau[0]/s)

In [ ]:
L_1_f_norm(f, model.B.detach().numpy(), model.Tau.detach().numpy(), 
           data_model.B_t, data_model.tau_t, N = 10000)

In [ ]:
print(loss(X_T - Y_T, D_T, data_model.B_t/np.sqrt(s), data_model.tau_t/s, Type = model_type).item())
print(loss(X_T - Y_T, D_T, model.B, model.Tau, Type = model_type).item())
print(model.loss_history[-1].item())

In [ ]:
print(loss(X_T - Y_T, D_T, data_model.B_t/data_model.tau_t[0], np.zeros(1), Type = model_type).item())
print(loss(X_T - Y_T, D_T, model.B/model.Tau[0], np.zeros(1), Type = model_type).item())

In [ ]:
Pred_test = pred(X_test, Y_test, data_model.B_t, np.array(tau))
print("test accuracy with noise (Using ground truth) = ", (Pred_test == D_test).sum().item()/D_test.shape[0])
print("test accuracy without noise (Using ground truth) = ", (Pred_test == D_no_noise_test).sum().item()/D_no_noise_test.shape[0])

In [ ]:
Pred_test = pred(X_test, Y_test, model.B, model.Tau)
print("test accuracy with noise", (Pred_test == D_test).sum().item()/D_test.shape[0])
print("test accuracy without noise", (Pred_test == D_no_noise_test).sum().item()/D_no_noise_test.shape[0])

In [ ]:
U_hat, S_hat, V_hat = LA.svd(M_hat_normal.detach().numpy(), full_matrices=False)

In [ ]:
U_t, S_t, V_t = LA.svd(M_t_normal, full_matrices=False)

In [ ]:
S_hat.sum(), S_t.sum()

In [ ]:
S_hat

In [ ]:
S_t

In [ ]:
plt.plot(S_hat, label = 'eigenvalues of M_hat/tau_hat', color= 'red')
plt.plot(S_t, label = 'eigenvalues of M_star/tau_star', color = 'blue')
plt.legend()
plt.xlabel('#eigenvalue', fontsize=10)
plt.ylabel('eigenvalue', fontsize=10)
plt.title('eigenvalues of  M_hat/tau_hat and M_star/tau_star' + title_suffix, fontsize= 12)
plt.savefig(image_path + 'eigen')
plt.show()

In [ ]:
# np.savetxt(csv_path + "eigen_S_star_normal.csv", 
#            S_t,
#            delimiter =", ", 
#            fmt ='% s')

In [ ]:
np.savetxt(csv_path + "eigen.csv", 
           S_hat,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
H = []
m = 7
for i in range(m):
    H.append(proj(U_hat[:, i], U_t[:, :m]))
    
plt.plot([j+1 for j in range(m)], H, label = '1 - Cosine')
plt.legend()
plt.savefig(image_path + "Cosine")
plt.show()

In [ ]:
np.savetxt(csv_path + "Cosine.csv", 
           H,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
for i in range(d):
    print(np.dot(U_hat[:,i], U_t[:,i]))

In [ ]:
np.diag(U_hat.T @ U_t)

In [ ]:
I = model.epoch_history
plt.plot(I, model.train_accuracy_with_noise, label = 'train_accuracy_with_noise', color = 'black')
plt.plot(I, model.train_accuracy_without_noise, label = 'train_accuracy_without_noise', color = 'red')
plt.plot(I, model.test_accuracy_with_noise, label = 'test_accuracy_with_noise', color = 'green')
plt.plot(I, model.test_accuracy_without_noise, label = 'test_accuracy_without_noise', color = 'blue')
plt.legend()

plt.xlabel('epoch', fontsize=10)
plt.ylabel('accuracy', fontsize=10)
plt.title('Test and Train accuracy vs epochs' + title_suffix, fontsize=12)
plt.savefig(image_path + "Test_and_Train_accuracy_vs_epochs")
plt.show()

In [ ]:
np.savetxt(csv_path + "model_epoch_history.csv", 
           model.epoch_history,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
np.savetxt(csv_path + "model_train_accuracy_with_noise.csv", 
           model.train_accuracy_with_noise,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
np.savetxt(csv_path + "model_train_accuracy_without_noise.csv", 
           model.train_accuracy_without_noise,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
np.savetxt(csv_path + "model_test_accuracy_with_noise.csv", 
           model.test_accuracy_with_noise,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
np.savetxt(csv_path + "model_test_accuracy_without_noise.csv", 
           model.test_accuracy_without_noise,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
model.loss_history[-1].item()

In [ ]:
# loss(X_T - Y_T, D_T, model.B, model.Tau).item()
l_value = loss(X_T - Y_T, D_T, data_model.B_t/np.sqrt(s), data_model.tau_t/s, Type = model_type).item()
I_ = np.array([l_value for _ in I])

In [ ]:
plt.plot(I[:], model.loss_history[:], label = 'loss_history', color = 'blue')
plt.plot(I[:], I_[:], label = 'loss_on_M_star', color = 'red')
plt.legend()
plt.xlabel('# iteration', fontsize=10)
plt.ylabel('loss', fontsize=10)
plt.title('loss_history vs iteration' + title_suffix, fontsize=12)
plt.savefig(image_path + "loss_history_vs_iteration")
plt.show()

In [ ]:
np.savetxt(csv_path + "model.loss_history.csv", 
           model.loss_history,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
np.savetxt(csv_path + "true_loss_history.csv", 
           I_,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
model.L_1_f_norm_history[-1]

In [ ]:
plt.plot(I[:], model.L_1_f_norm_history[:], label = 'L_1_f_norm_history', color = 'blue')
plt.legend()
plt.xlabel('# iteration', fontsize=10)
plt.ylabel('norm_history', fontsize=10)
plt.title('L_1_f_norm_history vs iteration'+ title_suffix, fontsize=12)
plt.savefig(image_path + "_norm_history_vs_iteration")
plt.show()

In [ ]:
np.savetxt(csv_path + "model_L_1_f_norm_history.csv", 
           model.L_1_f_norm_history,
           delimiter =", ", 
           fmt ='% s')

## Sample complexity

In [ ]:
d = 10
k = 10
n_labels = len(tau)+1

In [ ]:
H = []
J = []
N = 0
while N <= 10000:
    if N < 100:
        N += 10
    elif N < 1000:
        N += 100
    elif N < 5000:
        N+= 1000
    else: N += 5000
    
    
    X, Y, D_noisy, D_no_noisy = data_model.generate(f, 
                                                    N + 2000, 
                                                    tau = np.array(tau), 
                                                    noise_type = noise_type, 
                                                    noise_par = std)
    X = torch.tensor(X, dtype = torch.float64)
    Y = torch.tensor(Y, dtype = torch.float64)
    D_noisy = torch.tensor(D_noisy, dtype = torch.torch.int64)
    D_no_noisy = torch.tensor(D_no_noisy, dtype = torch.torch.int64)
        
    print('round for N = {} has started'.format(N))
    
    J.append(N)
    X, Y, D, X_T, Y_T, D_T, D_no_noise_T, X_test, Y_test, D_test, D_no_noise_test = train_test_split(X, 
                                                                                                     Y, 
                                                                                                 D_noisy, 
                                                                                                 D_no_noisy, 
                                                                                                 n_train = N)
    model = ML(d, k, n_labels, 
               X_T, Y_T, 
               D_T, D_no_noise_T, 
               X_test, Y_test, 
               D_test, D_no_noise_test, Type = model_type) 
    
    model.train(learning_rate = 2e-1, 
                n_iters = 30000, 
                decay = .99, 
                show_log = False)
    H.append(model)

In [ ]:
H[-1].accuracy(X_new, Y_new, D_new)

In [ ]:
test_accuracy_no_noise  = [H[i].accuracy(X_new, Y_new, D_new) for i in range(len(H))]

In [ ]:
plt.plot(J, test_accuracy_no_noise, label = 'test_accuracy (no noise)', color = 'blue')
plt.legend()
plt.xlabel('# samples', fontsize=10)
plt.ylabel('accuracy', fontsize=10)
plt.title('test_accuracy (no noise) vs sample complexity' + title_suffix, fontsize=12)
plt.savefig(image_path + "test_accuracy_(no_noise)_vs_sample_complexity")
plt.show()

In [ ]:
np.savetxt(csv_path + "test_accuracy_no_noise.csv", 
           test_accuracy_no_noise,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
np.savetxt(csv_path + "number_samples_list.csv", 
           J,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
test_accuracy_noisy  = [H[i].accuracy(H[i].X_test, H[i].Y_test, H[i].D_test) for i in range(len(H))]

In [ ]:
plt.plot(J, test_accuracy_noisy, label = 'test_accuracy(noisy)', color = 'blue')
plt.legend()
plt.xlabel('# samples', fontsize=10)
plt.ylabel('accuracy', fontsize=10)
plt.title('test_accuracy (noisy) vs sample complexity'+ title_suffix, fontsize=10)
plt.savefig(image_path + 'test_accuracy_(noisy)_vs_sample_complexity')
plt.show()

In [ ]:
np.savetxt(csv_path + "test_accuracy_noisy.csv", 
           test_accuracy_noisy,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
train_accuracy_noisy  = [H[i].accuracy(H[i].X_T, H[i].Y_T, H[i].D_T) for i in range(len(H))]

In [ ]:
plt.plot(J, train_accuracy_noisy, label = 'train_accuracy (noisy)', color = 'blue')
plt.legend()
plt.xlabel('# samples', fontsize=10)
plt.ylabel('accuracy', fontsize=10)
plt.title('train_accuracy (noisy) vs sample complexity' + title_suffix, fontsize=12)
plt.savefig(image_path + "train_accuracy_(noisy)_vs_sample_complexity")
plt.show()

In [ ]:
np.savetxt(csv_path + "train_accuracy_noisy.csv", 
           train_accuracy_noisy,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
train_accuracy_no_noisy  = [H[i].accuracy(H[i].X_T, H[i].Y_T, H[i].D_no_noise_T) for i in range(len(H))]

In [ ]:
plt.plot(J, train_accuracy_no_noisy, label = 'train_accuracy (no noise)', color = 'blue')
plt.legend()
plt.xlabel('# samples', fontsize=10)
plt.ylabel('accuracy', fontsize=10)
plt.title('train_accuracy (no noise) vs sample complexity' +  title_suffix, fontsize=12)
plt.savefig(image_path + "train_accuracy_(no_noise)_vs_sample_complexity")
plt.show()

In [ ]:
np.savetxt(csv_path + "train_accuracy_no_noisy.csv", 
           train_accuracy_no_noisy,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
plt.plot(J, train_accuracy_no_noisy, label = 'train_accuracy (no noise)', color = 'blue')
plt.plot(J, train_accuracy_noisy, label = 'train_accuracy (noisy)', color = 'red')
plt.plot(J, test_accuracy_noisy, label = 'test_accuracy(noisy)', color = 'green')
plt.plot(J, test_accuracy_no_noise, label = 'test_accuracy (no noise)', color = 'magenta')

plt.legend()
plt.xlabel('# samples', fontsize=10)
plt.ylabel('accuracy', fontsize=10)
plt.title('accuracy vs sample complexity' + title_suffix, fontsize=12)
plt.savefig(image_path + "accuracy_vs_sample_complexity")
plt.show()

In [ ]:
I = H[-1].epoch_history[:]
plt.plot(I, H[-1].train_accuracy_with_noise[:], label = 'train_accuracy_with_noise', color = 'black')
plt.plot(I, H[-1].train_accuracy_without_noise[:], label = 'train_accuracy_without_noise', color = 'red')
plt.plot(I, H[-1].test_accuracy_with_noise[:], label = 'test_accuracy_with_noise', color = 'green')
plt.plot(I, H[-1].test_accuracy_without_noise[:], label = 'test_accuracy_without_noise', color = 'blue')
plt.legend()
plt.xlabel('# Epoch', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy vs Epoch' + title_suffix, fontsize=12)
plt.savefig(image_path + 'Accuracy_vs_Epoch')
plt.show()

### How much noise breaks the model!

In [ ]:
noise_par_list = [i/10 for i in range(11)] + [1+ i/5 for i in range(1,6)] + [2.5, 4, 50]

In [ ]:
std_list = [s_ * np.pi/np.sqrt(3) for s_ in noise_par_list]

In [ ]:
print(tau)
N = 20000

In [ ]:
X, Y, D_noisy, D_no_noisy = data_model.generate(f, N = 20000, tau = np.array(tau), 
                                          noise_type = noise_type, noise_par = 40)
noise_amount = (D_noisy != D_no_noisy).mean()
print('the amount of noise: {}'.format(noise_amount))

In [ ]:
(D_no_noisy == 0).sum(), (D_no_noisy == 1).sum(), (D_no_noisy == 1).sum()/N

In [ ]:
# noise_par_list = [0, .2, .5, .8, 1.1, 1.4, 1.85, 2., 2.3, 2.5, 2.8, 2.9, 3.1,
#  3.9, 4.1, 4.35, 4.4, 4.5, 4.6, 4.8, 7.5, 12.5, 27, 60, 80, 150]

In [ ]:
# np.savetxt("CSVs/Gaussian/scale_list.csv", 
#            noise_par_list,
#            delimiter =", ", 
#            fmt ='% s')

In [ ]:
N=20000
d = 10
k = 10
n_labels = len(tau) + 1
noise_list_history = []
model_history = []
for std_ in std_list:
    X, Y, D_noisy, D_no_noisy = data_model.generate(f, N, tau = np.array(tau), 
                                                    noise_type = noise_type, noise_par = std_)
    noise_amount = (D_noisy != D_no_noisy).mean()
    print('the amount of noise: {}'.format(noise_amount))
    noise_list_history.append(noise_amount)
    
    X = torch.tensor(X, dtype = torch.float64)
    Y = torch.tensor(Y, dtype = torch.float64)
    D_noisy = torch.tensor(D_noisy, dtype = torch.torch.int64)
    D_no_noisy = torch.tensor(D_no_noisy, dtype = torch.torch.int64)
    
    X,Y,D,X_T, Y_T,D_T,D_no_noise_T, X_test,Y_test,D_test,D_no_noise_test = train_test_split(X,
                                                                                             Y, 
                                                                                             D_noisy, 
                                                                                             D_no_noisy,
                                                                                             n_train = 18000)
    model = ML(d, k, n_labels, 
               X_T, Y_T, D_T, D_no_noise_T,
               X_test, Y_test, D_test, D_no_noise_test, Type = model_type)
    
    model.train(learning_rate = 1e-1, 
                n_iters = 30001, 
                decay = .95, 
                show_log = False)
    
    model_history.append(model)
    
    print('train_accuracy (noisey) = {}'.format(model_history[-1].accuracy(model_history[-1].X_T, 
                                                                           model_history[-1].Y_T, 
                                                                           model_history[-1].D_T)))
    
    print('test_accuracy (noisey) = {}'.format(model_history[-1].accuracy(model_history[-1].X_test, 
                                                                          model_history[-1].Y_test, 
                                                                          model_history[-1].D_test)))
    
    print('train_accuracy (no noise) = {}'.format(model_history[-1].accuracy(model_history[-1].X_T, 
                                                                             model_history[-1].Y_T, 
                                                                             model_history[-1].D_no_noise_T)))
    
    print('test_accuracy (no noise) = {}'.format(model_history[-1].accuracy(model_history[-1].X_test, 
                                                                            model_history[-1].Y_test, 
                                                                            model_history[-1].D_no_noise_test)))
    print('\n')

In [ ]:
np.savetxt(csv_path + "noise_list_history.csv", 
           noise_list_history,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
plt.plot(noise_par_list, noise_list_history, label = 'variance VS noise', color = 'blue')
plt.legend()
plt.ylabel('# Noise', fontsize = 10)
plt.xlabel('variance', fontsize = 10)
plt.title('Noise vs variance [Gaussian]', fontsize = 10)
plt.show()

In [ ]:
X_new, Y_new, _, D_new = data_model.generate(f, 
                                             N = 10000, 
                                             tau = np.array(tau), 
                                             noise_type = None, 
                                             noise_par = None)
X_new = torch.tensor(X_new, dtype = torch.float64)
Y_new = torch.tensor(Y_new, dtype = torch.float64)
D_new = torch.tensor(D_new, dtype = torch.torch.int64)

In [ ]:
test_accuracy_no_noise  = [model_history[i].accuracy(X_new, Y_new, D_new) for i in range(len(model_history))]

plt.plot(noise_list_history, test_accuracy_no_noise, label = 'test_accuracy (no noise)', color = 'blue')
plt.legend()
plt.xlabel('noise', fontsize = 10)
plt.ylabel('accuracy', fontsize = 10)
plt.title('test_accuracy (no noise) vs noise' + title_suffix, fontsize = 12)
plt.savefig(image_path + 'Test_accuracy_(no_noise)_vs_noise')
plt.show()

In [ ]:
np.savetxt(csv_path + "test_accuracy_no_noise.csv", 
           test_accuracy_no_noise,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
[(noise_list_history[i], test_accuracy_no_noise[i]) for i in range(len(noise_list_history))]

In [ ]:
test_accuracy_noisy  = [model_history[i].accuracy(model_history[i].X_test, 
                                                  model_history[i].Y_test, 
                                                  model_history[i].D_test) for i in range(len(model_history))]

plt.plot(noise_list_history, test_accuracy_noisy, label = 'test_accuracy(noisy)', color = 'blue')
plt.legend()
plt.xlabel('noise', fontsize = 10)
plt.ylabel('accuracy', fontsize = 10)
plt.title('test_accuracy (noisy) vs noise' + title_suffix, fontsize = 12)
plt.savefig(image_path + 'test_accuracy_(noisy)_vs_noise')
plt.show()

In [ ]:
np.savetxt(csv_path + "test_accuracy_noisy.csv", 
           test_accuracy_noisy,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
train_accuracy_noisy  = [model_history[i].accuracy(model_history[i].X_T, 
                                                   model_history[i].Y_T, 
                                                   model_history[i].D_T) for i in range(len(model_history))]

plt.plot(noise_list_history, train_accuracy_noisy, label = 'train_accuracy (noisy)', color = 'blue')
plt.legend()
plt.xlabel('noise', fontsize = 10)
plt.ylabel('accuracy', fontsize = 10)
plt.title('train_accuracy (noisy) vs noise' + title_suffix, fontsize = 10)
plt.savefig(image_path + 'train_accuracy_(noisy)_vs_noise')
plt.show()

In [ ]:
np.savetxt(csv_path + "train_accuracy_noisy.csv", 
           train_accuracy_noisy,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
train_accuracy_no_noisy  = [model_history[i].accuracy(model_history[i].X_T, 
                                                      model_history[i].Y_T, 
                                                      model_history[i].D_no_noise_T) for i in range(len(model_history))]
plt.plot(noise_list_history, 
         train_accuracy_no_noisy, 
         label = 'train_accuracy (no noise)', 
         color = 'blue')
plt.legend()
plt.xlabel('noise', fontsize = 10)
plt.ylabel('accuracy', fontsize = 10)
plt.title('train_accuracy (no noise) vs noise' + title_suffix, fontsize = 12)
plt.savefig(image_path + 'train_accuracy_(no_noise)_vs_noise')
plt.show()

In [ ]:
np.savetxt(csv_path + "train_accuracy_no_noisy.csv", 
           train_accuracy_no_noisy,
           delimiter =", ", 
           fmt ='% s')

In [ ]:
noise_list_history[-1], train_accuracy_no_noisy[-1]